In [1]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd


In [6]:
df = pd.read_csv("../Data/Raw/new_approach/Weekly_Covid_Data_Population_Normalized.csv")

In [7]:
print(df.columns)
print(df.shape)

Index(['week_no', 'iso_code', 'new_cases', 'new_deaths', 'new_vaccinations',
       'new_people_vaccinated', 'reproduction_rate', 'stringency_index',
       'excess_mortality', 'population_density', 'median_age', 'aged_65_older',
       'aged_70_older', 'cardiovasc_death_rate', 'diabetes_prevalence',
       'female_smokers', 'male_smokers', 'hospital_beds_per_thousand',
       'life_expectancy', 'human_development_index', 'population'],
      dtype='object')
(41184, 21)


In [9]:

df.drop(["iso_code", "week_no"], axis=1, inplace=True)
#df.drop(columns=df.columns[0:1], axis=1, inplace=True)
pop = df.iloc[0]["population"]
last_country = df.iloc[234*176 - 1]
print(last_country)
cnt = df["population"].value_counts()

new_cases                     4.595436e-06
new_deaths                    1.225450e-07
new_vaccinations              0.000000e+00
new_people_vaccinated         0.000000e+00
reproduction_rate             9.500000e-01
stringency_index              8.796000e+01
excess_mortality             -1.000000e+03
population_density            4.272900e+01
median_age                    1.960000e+01
aged_65_older                 2.822000e+00
aged_70_older                 1.882000e+00
cardiovasc_death_rate         3.078460e+02
diabetes_prevalence           1.820000e+00
female_smokers                1.600000e+00
male_smokers                  3.070000e+01
hospital_beds_per_thousand    1.700000e+00
life_expectancy               6.149000e+01
human_development_index       5.710000e-01
population                    1.632054e+07
Name: 41183, dtype: float64


In [11]:
# Here we are going to normilze the data.
from sklearn import preprocessing

x = df.values
min_max_scaler = preprocessing.MinMaxScaler()
x_scaled = min_max_scaler.fit_transform(x)
df = pd.DataFrame(x_scaled)

In [12]:
# now we want to join datas together
prediction_days_count = 2
features_list = []
X_list = []
Y_list = []
for i in range(234): # 234 is number of countries
    for j in range(176): # 176 is number of weeks we have
        row_list = df.loc[i*176 + j, :].values.flatten().tolist()
        features_list.append(row_list)
# print(len(features_list))
# print((len(features_list) - prediction_days_count) + 1)
for i in range((len(features_list) - prediction_days_count) + 1):
    fl = features_list[i]
    fl_len = len(features_list[i])
    this_population = features_list[i][fl_len - 1]
    for j in range(1, prediction_days_count):
        if this_population == features_list[i + j][fl_len - 1]:
            fl.extend(features_list[i + j])
        else:
            this_population = -1
            break
    if this_population != -1:
        X_list.append(fl)
        Y_list.append(features_list[(i + prediction_days_count) - 1][0])


In [15]:
print(len(X_list))
print(len(Y_list))
np.shape(X_list)

40950
40950


(40950, 38)

In [16]:
import math
def train_test_splitter(X, Y, prediction_days_count, batch_count, batch_length):
    xlen = len(X)
    xs_count_for_each_country = int(xlen / 234)
    X_train = []
    X_test = []
    Y_train = []
    Y_test = []
    steps = math.floor((xs_count_for_each_country - (batch_count * batch_length)) / (batch_count + 1)) 
    i = 0
    #print(steps)
    for i in range(234):
        j = 0
        for j in range((i * xs_count_for_each_country),(i * xs_count_for_each_country) + steps):
            X_train.append(X[j])
        #print(j)
        while(j + batch_length + steps < (i+1) * xs_count_for_each_country):
            for k in range(j + prediction_days_count, j + batch_length - prediction_days_count):
                X_test.append(X[j])
                Y_test.append(Y[j])
            j += batch_length
            for k in range(j, j + steps):
                X_train.append(X[j])
                Y_train.append(Y[j])
            j += steps + 1
            while j < (i+1) * xs_count_for_each_country:
                X_train.append(X[j])
                Y_train.append(Y[j])
                j += 1
        #print(j)
    ########
    # for i in range(steps):
    #     X_train.append(X[i])
    # while(i + batch_length + steps < xlen):
    #     for j in range(batch_length):
    #         X_test.append(X[j])
    #         Y_test.append(Y[j])
    #     for j in range(i, i + steps):
    #         X_train.append(X[j])
    #         Y_train.append(Y[j])
    #     i += batch_length + steps
    return X_train, X_test, Y_train, Y_test
        

In [17]:
X_train, X_test, Y_train, Y_test = train_test_splitter(X_list, Y_list, prediction_days_count, 3, 10)


In [19]:
print(np.shape(X_train))
print(len(X_train) + len(X_test))
print(X_train[0])

(38610, 38)
40014
[0.0, 0.0, 0.9135528498199288, 0.9135528498199288, 0.9963668939431642, 0.9090909090909091, 0.0, 0.0518207150755621, 0.9717611142911657, 0.9761764044363999, 0.9831555052415677, 0.9261269170971986, 0.9796803586504031, 0.0, 0.0, 0.9868810416255672, 0.9798297676558546, 0.9995544264139219, 0.02884430250905809, 0.0, 0.0, 0.9135528498199288, 0.9135528498199288, 0.9963668939431642, 0.9090909090909091, 0.0, 0.0518207150755621, 0.9717611142911657, 0.9761764044363999, 0.9831555052415677, 0.9261269170971986, 0.9796803586504031, 0.0, 0.0, 0.9868810416255672, 0.9798297676558546, 0.9995544264139219, 0.02884430250905809]


In [ ]:
# ns_list, train_scores, validation_scores = learning_curve(
#                                                    estimator = SVR(**opt_svr_param), 
#                                                    X = X, y = Y, 
#                                                    train_sizes = ns_list, cv = 5,
#                                                    scoring = 'neg_mean_squared_error')


In [ ]:
# train_scores, valid_scores = validation_curve(KNeighborsRegressor(), X, Y, 
#                                               param_name="n_neighbors",
#                                               param_range=k_list , cv=20, 
#                                               scoring = 'neg_mean_squared_error',
#                                               verbose=1, n_jobs=-1
#                                              )
